In [69]:
import numpy as np

i1 = np.array([1, 5, 3, 4])
num_m1 = np.array([1, 1, 1, 1])
val1 = np.array([0.2, 0.3, 0.4, 0.5])

i2 = np.array([2, 7, 8])
num_m2 = np.array([1, 1, 1])
val2 = np.array([1, 2, 3])




set(num_m1) == set(num_m2) and len(set(num_m1)) == 1



True

In [78]:
list1 = [1, 2, 3]
list2 = [4, 5, 6]
list1.extend(list2)
list1

[1, 2, 3, 4, 5, 6]

In [70]:
np.append(i1, i2, axis=0)

array([1, 5, 3, 4, 2, 7, 8])

In [71]:
def vectorized_operations_example(T1, T2, operation_func):
    """
    Instead of creating full Cartesian product, apply operations directly.
    Example: computing sum of all pairs without storing pairs.
    """
    # Broadcasting for direct computation
    T1_broadcast = T1[:, np.newaxis]
    T2_broadcast = T2[np.newaxis, :]
    
    # Apply operation directly on broadcasted arrays
    result = operation_func(T1_broadcast, T2_broadcast)
    return result

val = vectorized_operations_example(val1, val2, np.multiply)
val = val.flatten()
print("val :", val)
i_j = vectorized_operations_example(i2, i1, np.meshgrid)
i_j = np.array(i_j).flatten()
print("val .shape :     ", val.shape)
print("i :", i_j[:val.shape[0]])
print("j : ", i_j[val.shape[0]:])
i = i_j[:val.shape[0]]
j = i_j[val.shape[0]:]
num_m = np.broadcast_to(num_m1[:, np.newaxis], (len(num_m1), len(num_m2))).flatten()
print("num_m :", num_m)

val : [0.2 0.4 0.6 0.3 0.6 0.9 0.4 0.8 1.2 0.5 1.  1.5]
val .shape :      (12,)
i : [2 7 8 2 7 8 2 7 8 2 7 8]
j :  [1 1 1 5 5 5 3 3 3 4 4 4]
num_m : [1 1 1 1 1 1 1 1 1 1 1 1]


In [75]:
i = np.append(i, 2)
j = np.append(j, 1)
num_m = np.append(num_m, 1)
val = np.append(val, 99)

In [76]:

# Example: T1, T2, T3, T4 are 1D arrays of the same length
# Combine T1, T2, T3 into a structured array of tuples
print("i :", i)
print("j :", j)
i_ = np.where(i > j , i, j)
j_ = np.where(i > j , j, i)
print("i :", i_)
print("j :", j_)
keys = np.stack((i_, j_, num_m), axis=-1)

# Get unique keys and inverse indices
unique_keys, inverse = np.unique(keys, axis=0, return_inverse=True)

# Sum T4 values for each unique key
summed_T4 = np.bincount(inverse, weights=val)

# Split unique keys back into T1, T2, T3
T1_red, T2_red, T3_red = unique_keys.T
T4_red = summed_T4

print("T1_red :", T1_red)
print("T2_red :", T2_red)
print("T3_red :", T3_red)
print("T4_red :", T4_red)

i : [2 7 8 2 7 8 2 7 8 2 7 8 2]
j : [1 1 1 5 5 5 3 3 3 4 4 4 1]
i : [2 7 8 5 7 8 3 7 8 4 7 8 2]
j : [1 1 1 2 5 5 2 3 3 2 4 4 1]
T1_red : [2 3 4 5 7 7 7 7 8 8 8 8]
T2_red : [1 2 2 2 1 3 4 5 1 3 4 5]
T3_red : [1 1 1 1 1 1 1 1 1 1 1 1]
T4_red : [99.2  0.4  0.5  0.3  0.4  0.8  1.   0.6  0.6  1.2  1.5  0.9]


In [52]:
T1_red

array([6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8])

In [40]:
T1 = np.repeat(i1, len(i2))
T2 = np.tile(i2, len(i1))
val = np.repeat(val1, len(i2))* (np.tile(val2, len(i1)))

print("T1 :", T1)
print("T2 :", T2)
print("val :", val)

T1 : [1 1 1 5 5 5 3 3 3 4 4 4]
T2 : [6 7 8 6 7 8 6 7 8 6 7 8]
val : [0.2 0.4 0.6 0.3 0.6 0.9 0.4 0.8 1.2 0.5 1.  1.5]


In [ ]:
import numpy as np
import itertools
from numba import jit
import time

# ============ FASTER ALTERNATIVES ============

def lazy_cartesian_iterator(T1, T2):
    """
    FASTEST for sequential access - no memory allocation upfront.
    Memory: O(1), Time per element: O(1)
    """
    for t1 in T1:
        for t2 in T2:
            yield t1, t2

def itertools_cartesian(T1, T2):
    """
    Using itertools.product - very memory efficient.
    """
    return itertools.product(T1, T2)

@jit(nopython=True)
def numba_cartesian_fill(T1, T2, i1, i2):
    """
    Numba-compiled version - much faster for large arrays.
    Pre-allocate arrays and fill them with compiled code.
    """
    idx = 0
    for i in range(len(T1)):
        for j in range(len(T2)):
            i1[idx] = T1[i]
            i2[idx] = T2[j]
            idx += 1

def fast_cartesian_numba(T1, T2):
    """
    Fastest for large arrays when you need materialized result.
    """
    T1 = np.asarray(T1)
    T2 = np.asarray(T2)
    
    total_size = len(T1) * len(T2)
    i1 = np.empty(total_size, dtype=T1.dtype)
    i2 = np.empty(total_size, dtype=T2.dtype)
    
    numba_cartesian_fill(T1, T2, i1, i2)
    return i1, i2

def broadcasting_indices(T1, T2):import numpy as np
import itertools
from numba import jit
import time

# ============ FASTER ALTERNATIVES ============

def lazy_cartesian_iterator(T1, T2):
    """
    FASTEST for sequential access - no memory allocation upfront.
    Memory: O(1), Time per element: O(1)
    """
    for t1 in T1:
        for t2 in T2:
            yield t1, t2

def itertools_cartesian(T1, T2):
    """
    Using itertools.product - very memory efficient.
    """
    return itertools.product(T1, T2)

@jit(nopython=True)
def numba_cartesian_fill(T1, T2, i1, i2):
    """
    Numba-compiled version - much faster for large arrays.
    Pre-allocate arrays and fill them with compiled code.
    """
    idx = 0
    for i in range(len(T1)):
        for j in range(len(T2)):
            i1[idx] = T1[i]
            i2[idx] = T2[j]
            idx += 1

def fast_cartesian_numba(T1, T2):
    """
    Fastest for large arrays when you need materialized result.
    """
    T1 = np.asarray(T1)
    T2 = np.asarray(T2)
    
    total_size = len(T1) * len(T2)
    i1 = np.empty(total_size, dtype=T1.dtype)
    i2 = np.empty(total_size, dtype=T2.dtype)
    
    numba_cartesian_fill(T1, T2, i1, i2)
    return i1, i2

def broadcasting_indices(T1, T2):
    """
    Use broadcasting with indices - avoids copying data.
    Most memory efficient when you can work with indices.
    """
    i_indices = np.arange(len(T1))[:, None]
    j_indices = np.arange(len(T2))[None, :]
    
    # Return indices that can be used to access T1 and T2
    return i_indices + np.zeros_like(j_indices), j_indices + np.zeros_like(i_indices)

def chunked_cartesian(T1, T2, chunk_size=1000):
    """
    Process Cartesian product in chunks - memory efficient for huge arrays.
    """
    T1 = np.asarray(T1)
    T2 = np.asarray(T2)
    
    for i in range(0, len(T1), chunk_size):
        chunk_T1 = T1[i:i+chunk_size]
        i1_chunk = np.repeat(chunk_T1, len(T2))
        i2_chunk = np.tile(T2, len(chunk_T1))
        yield i1_chunk, i2_chunk

def meshgrid_view_based(T1, T2):
    """
    Use meshgrid but delay flattening - work with 2D views when possible.
    """
    mesh1, mesh2 = np.meshgrid(T1, T2, indexing='ij')
    return mesh1, mesh2  # Return 2D arrays instead of flattening

def vectorized_operations_example(T1, T2, operation_func):
    """
    Instead of creating full Cartesian product, apply operations directly.
    Example: computing sum of all pairs without storing pairs.
    """
    # Broadcasting for direct computation
    T1_broadcast = T1[:, np.newaxis]
    T2_broadcast = T2[np.newaxis, :]
    
    # Apply operation directly on broadcasted arrays
    result = operation_func(T1_broadcast, T2_broadcast)
    return result

# ============ SPECIALIZED FAST IMPLEMENTATIONS ============

def fast_cartesian_memoryview(T1, T2):
    """
    Using memory views for even faster access (when using Cython/Numba).
    """
    T1 = np.asarray(T1)
    T2 = np.asarray(T2)
    
    total_size = len(T1) * len(T2)
    i1 = np.empty(total_size, dtype=T1.dtype)
    i2 = np.empty(total_size, dtype=T2.dtype)
    
    # Manual filling (would be faster in Cython)
    idx = 0
    for t1 in T1:
        for t2 in T2:
            i1[idx] = t1
            i2[idx] = t2
            idx += 1
    
    return i1, i2

def preallocated_cartesian(T1, T2):
    """
    Pre-allocate arrays with correct dtype for better performance.
    """
    T1 = np.asarray(T1)
    T2 = np.asarray(T2)
    
    total_size = len(T1) * len(T2)
    
    # Pre-allocate with specific dtypes
    i1 = np.empty(total_size, dtype=T1.dtype)
    i2 = np.empty(total_size, dtype=T2.dtype)
    
    # Use numpy operations on pre-allocated arrays
    i1[:] = np.repeat(T1, len(T2))
    i2[:] = np.tile(T2, len(T1))
    
    return i1, i2

# ============ BENCHMARKING ============

def benchmark_all_methods(T1, T2, iterations=100):
    """
    Benchmark all methods comprehensively.
    """
    methods = [
        ("Standard repeat+tile", lambda: (np.repeat(T1, len(T2)), np.tile(T2, len(T1)))),
        ("Numba compiled", lambda: fast_cartesian_numba(T1, T2)),
        ("Pre-allocated", lambda: preallocated_cartesian(T1, T2)),
        ("Memory view style", lambda: fast_cartesian_memoryview(T1, T2)),
        ("Meshgrid+flatten", lambda: [x.flatten() for x in np.meshgrid(T1, T2, indexing='ij')]),
    ]
    
    print(f"Benchmarking with T1 size: {len(T1)}, T2 size: {len(T2)}")
    print(f"Output size: {len(T1) * len(T2)}, Iterations: {iterations}")
    print("-" * 60)
    
    results = {}
    
    for name, method in methods:
        # Warmup for Numba
        if "Numba" in name:
            method()
        
        start = time.time()
        for _ in range(iterations):
            result = method()
        elapsed = time.time() - start
        
        results[name] = elapsed / iterations
        print(f"{name:20}: {elapsed/iterations:.6f}s per call")
    
    return results

def memory_efficient_examples():
    """
    Show memory-efficient alternatives for different use cases.
    """
    print("\n=== MEMORY-EFFICIENT ALTERNATIVES ===")
    
    T1 = np.arange(1000)
    T2 = np.arange(2000)
    
    print(f"Input sizes: T1={len(T1)}, T2={len(T2)}")
    print(f"Full Cartesian product would need: {len(T1) * len(T2) * 8 / 1e6:.1f} MB")
    
    print("\n1. Lazy iteration (zero upfront memory):")
    lazy_iter = lazy_cartesian_iterator(T1[:10], T2[:10])
    print("   First 3 pairs:", [next(lazy_iter) for _ in range(3)])
    
    print("\n2. Chunked processing:")
    chunk_gen = chunked_cartesian(T1[:100], T2[:100], chunk_size=50)
    first_chunk = next(chunk_gen)
    print(f"   First chunk size: {len(first_chunk[0])}")
    
    print("\n3. Broadcasting for direct computation:")
    # Example: compute sum of all pairs without storing pairs
    sum_matrix = vectorized_operations_example(T1[:10], T2[:10], np.add)
    print(f"   Sum matrix shape: {sum_matrix.shape}")
    print(f"   Memory saved: {100 * (1 - sum_matrix.nbytes / (len(T1[:10]) * len(T2[:10]) * 2 * 8)):.1f}%")

def speed_comparison_demo():
    """
    Demonstrate speed improvements with different array sizes.
    """
    print("\n=== SPEED COMPARISON ===")
    
    sizes = [(100, 100), (500, 200), (1000, 100)]
    
    for t1_size, t2_size in sizes:
        print(f"\nArray sizes: T1={t1_size}, T2={t2_size}")
        T1 = np.arange(t1_size)
        T2 = np.arange(t2_size)
        
        # Standard method
        start = time.time()
        i1_std, i2_std = np.repeat(T1, len(T2)), np.tile(T2, len(T1))
        std_time = time.time() - start
        
        # Numba method
        start = time.time()
        i1_numba, i2_numba = fast_cartesian_numba(T1, T2)
        numba_time = time.time() - start
        
        # Verify results are identical
        identical = np.array_equal(i1_std, i1_numba) and np.array_equal(i2_std, i2_numba)
        speedup = std_time / numba_time if numba_time > 0 else float('inf')
        
        print(f"  Standard: {std_time:.6f}s")
        print(f"  Numba:    {numba_time:.6f}s")
        print(f"  Speedup:  {speedup:.1f}x")
        print(f"  Results identical: {identical}")

if __name__ == "__main__":
    # Test with small arrays first
    T1 = [1, 2]
    T2 = [3, 4, 5]
    
    print("=== TESTING WITH SMALL ARRAYS ===")
    print(f"T1 = {T1}")
    print(f"T2 = {T2}")
    
    # Standard method
    i1_std, i2_std = np.repeat(T1, len(T2)), np.tile(T2, len(T1))
    print(f"\nStandard result:")
    print(f"i1 = {i1_std}")
    print(f"i2 = {i2_std}")
    
    # Fast numba method
    i1_fast, i2_fast = fast_cartesian_numba(T1, T2)
    print(f"\nNumba result:")
    print(f"i1 = {i1_fast}")
    print(f"i2 = {i2_fast}")
    
    # Verify they're the same
    print(f"Results identical: {np.array_equal(i1_std, i1_fast) and np.array_equal(i2_std, i2_fast)}")
    
    # Show lazy iteration
    print(f"\nLazy iteration:")
    for i, (a, b) in enumerate(lazy_cartesian_iterator(T1, T2)):
        print(f"  Pair {i}: ({a}, {b})")
    
    # Memory efficient examples
    memory_efficient_examples()
    
    # Speed comparison
    speed_comparison_demo()
    
    print("\n=== RECOMMENDATIONS BY USE CASE ===")
    print("• Sequential access only: Use lazy_cartesian_iterator() - O(1) memory")
    print("• Large arrays, need full result: Use fast_cartesian_numba() - 2-10x faster")
    print("• Memory constrained: Use chunked_cartesian() - process in batches")
    print("• Direct operations: Use broadcasting - avoid materialization entirely")
    print("• Small arrays: Standard repeat+tile is fine"
    """
    Use broadcasting with indices - avoids copying data.
    Most memory efficient when you can work with indices.
    """
    i_indices = np.arange(len(T1))[:, None]
    j_indices = np.arange(len(T2))[None, :]
    
    # Return indices that can be used to access T1 and T2
    return i_indices + np.zeros_like(j_indices), j_indices + np.zeros_like(i_indices)

def chunked_cartesian(T1, T2, chunk_size=1000):
    """
    Process Cartesian product in chunks - memory efficient for huge arrays.
    """
    T1 = np.asarray(T1)
    T2 = np.asarray(T2)
    
    for i in range(0, len(T1), chunk_size):
        chunk_T1 = T1[i:i+chunk_size]
        i1_chunk = np.repeat(chunk_T1, len(T2))
        i2_chunk = np.tile(T2, len(chunk_T1))
        yield i1_chunk, i2_chunk

def meshgrid_view_based(T1, T2):
    """
    Use meshgrid but delay flattening - work with 2D views when possible.
    """
    mesh1, mesh2 = np.meshgrid(T1, T2, indexing='ij')
    return mesh1, mesh2  # Return 2D arrays instead of flattening

def vectorized_operations_example(T1, T2, operation_func):
    """
    Instead of creating full Cartesian product, apply operations directly.
    Example: computing sum of all pairs without storing pairs.
    """
    # Broadcasting for direct computation
    T1_broadcast = T1[:, np.newaxis]
    T2_broadcast = T2[np.newaxis, :]
    
    # Apply operation directly on broadcasted arrays
    result = operation_func(T1_broadcast, T2_broadcast)
    return result

# ============ SPECIALIZED FAST IMPLEMENTATIONS ============

def fast_cartesian_memoryview(T1, T2):
    """
    Using memory views for even faster access (when using Cython/Numba).
    """
    T1 = np.asarray(T1)
    T2 = np.asarray(T2)
    
    total_size = len(T1) * len(T2)
    i1 = np.empty(total_size, dtype=T1.dtype)
    i2 = np.empty(total_size, dtype=T2.dtype)
    
    # Manual filling (would be faster in Cython)
    idx = 0
    for t1 in T1:
        for t2 in T2:
            i1[idx] = t1
            i2[idx] = t2
            idx += 1
    
    return i1, i2

def preallocated_cartesian(T1, T2):
    """
    Pre-allocate arrays with correct dtype for better performance.
    """
    T1 = np.asarray(T1)
    T2 = np.asarray(T2)
    
    total_size = len(T1) * len(T2)
    
    # Pre-allocate with specific dtypes
    i1 = np.empty(total_size, dtype=T1.dtype)
    i2 = np.empty(total_size, dtype=T2.dtype)
    
    # Use numpy operations on pre-allocated arrays
    i1[:] = np.repeat(T1, len(T2))
    i2[:] = np.tile(T2, len(T1))
    
    return i1, i2

# ============ BENCHMARKING ============

def benchmark_all_methods(T1, T2, iterations=100):
    """
    Benchmark all methods comprehensively.
    """
    methods = [
        ("Standard repeat+tile", lambda: (np.repeat(T1, len(T2)), np.tile(T2, len(T1)))),
        ("Numba compiled", lambda: fast_cartesian_numba(T1, T2)),
        ("Pre-allocated", lambda: preallocated_cartesian(T1, T2)),
        ("Memory view style", lambda: fast_cartesian_memoryview(T1, T2)),
        ("Meshgrid+flatten", lambda: [x.flatten() for x in np.meshgrid(T1, T2, indexing='ij')]),
    ]
    
    print(f"Benchmarking with T1 size: {len(T1)}, T2 size: {len(T2)}")
    print(f"Output size: {len(T1) * len(T2)}, Iterations: {iterations}")
    print("-" * 60)
    
    results = {}
    
    for name, method in methods:
        # Warmup for Numba
        if "Numba" in name:
            method()
        
        start = time.time()
        for _ in range(iterations):
            result = method()
        elapsed = time.time() - start
        
        results[name] = elapsed / iterations
        print(f"{name:20}: {elapsed/iterations:.6f}s per call")
    
    return results

def memory_efficient_examples():
    """
    Show memory-efficient alternatives for different use cases.
    """
    print("\n=== MEMORY-EFFICIENT ALTERNATIVES ===")
    
    T1 = np.arange(1000)
    T2 = np.arange(2000)
    
    print(f"Input sizes: T1={len(T1)}, T2={len(T2)}")
    print(f"Full Cartesian product would need: {len(T1) * len(T2) * 8 / 1e6:.1f} MB")
    
    print("\n1. Lazy iteration (zero upfront memory):")
    lazy_iter = lazy_cartesian_iterator(T1[:10], T2[:10])
    print("   First 3 pairs:", [next(lazy_iter) for _ in range(3)])
    
    print("\n2. Chunked processing:")
    chunk_gen = chunked_cartesian(T1[:100], T2[:100], chunk_size=50)
    first_chunk = next(chunk_gen)
    print(f"   First chunk size: {len(first_chunk[0])}")
    
    print("\n3. Broadcasting for direct computation:")
    # Example: compute sum of all pairs without storing pairs
    sum_matrix = vectorized_operations_example(T1[:10], T2[:10], np.add)
    print(f"   Sum matrix shape: {sum_matrix.shape}")
    print(f"   Memory saved: {100 * (1 - sum_matrix.nbytes / (len(T1[:10]) * len(T2[:10]) * 2 * 8)):.1f}%")

def speed_comparison_demo():
    """
    Demonstrate speed improvements with different array sizes.
    """
    print("\n=== SPEED COMPARISON ===")
    
    sizes = [(100, 100), (500, 200), (1000, 100)]
    
    for t1_size, t2_size in sizes:
        print(f"\nArray sizes: T1={t1_size}, T2={t2_size}")
        T1 = np.arange(t1_size)
        T2 = np.arange(t2_size)
        
        # Standard method
        start = time.time()
        i1_std, i2_std = np.repeat(T1, len(T2)), np.tile(T2, len(T1))
        std_time = time.time() - start
        
        # Numba method
        start = time.time()
        i1_numba, i2_numba = fast_cartesian_numba(T1, T2)
        numba_time = time.time() - start
        
        # Verify results are identical
        identical = np.array_equal(i1_std, i1_numba) and np.array_equal(i2_std, i2_numba)
        speedup = std_time / numba_time if numba_time > 0 else float('inf')
        
        print(f"  Standard: {std_time:.6f}s")
        print(f"  Numba:    {numba_time:.6f}s")
        print(f"  Speedup:  {speedup:.1f}x")
        print(f"  Results identical: {identical}")

if __name__ == "__main__":
    # Test with small arrays first
    T1 = [1, 2]
    T2 = [3, 4, 5]
    
    print("=== TESTING WITH SMALL ARRAYS ===")
    print(f"T1 = {T1}")
    print(f"T2 = {T2}")
    
    # Standard method
    i1_std, i2_std = np.repeat(T1, len(T2)), np.tile(T2, len(T1))
    print(f"\nStandard result:")
    print(f"i1 = {i1_std}")
    print(f"i2 = {i2_std}")
    
    # Fast numba method
    i1_fast, i2_fast = fast_cartesian_numba(T1, T2)
    print(f"\nNumba result:")
    print(f"i1 = {i1_fast}")
    print(f"i2 = {i2_fast}")
    
    # Verify they're the same
    print(f"Results identical: {np.array_equal(i1_std, i1_fast) and np.array_equal(i2_std, i2_fast)}")
    
    # Show lazy iteration
    print(f"\nLazy iteration:")
    for i, (a, b) in enumerate(lazy_cartesian_iterator(T1, T2)):
        print(f"  Pair {i}: ({a}, {b})")
    
    # Memory efficient examples
    memory_efficient_examples()
    
    # Speed comparison
    speed_comparison_demo()
    
    print("\n=== RECOMMENDATIONS BY USE CASE ===")
    print("• Sequential access only: Use lazy_cartesian_iterator() - O(1) memory")
    print("• Large arrays, need full result: Use fast_cartesian_numba() - 2-10x faster")
    print("• Memory constrained: Use chunked_cartesian() - process in batches")
    print("• Direct operations: Use broadcasting - avoid materialization entirely")
    print("• Small arrays: Standard repeat+tile is fine")

ModuleNotFoundError: No module named 'numba'

In [43]:
x = np.array([4, 5], dtype =float)

x.size


2

In [1]:
from numba import njit

@njit
def encode_key(i, j, k, J, K):
    return i * J * K + j * K + k


ModuleNotFoundError: No module named 'numba'